<a href="https://colab.research.google.com/github/kynthesis/HaystackResearch/blob/main/4_Preprocessing_Documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cách tiền xử lý văn bản**



# 1. Kiểm tra GPU runtime

In [1]:
%%bash

nvidia-smi

Sat Jul  1 12:48:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 2. Cài đặt Haystack

In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,ocr,preprocessing,file-conversion,pdf]

# 3. Bật chế độ logging

In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

# 4. Chuẩn bị các file tài liệu



In [4]:
from haystack.utils import fetch_archive_from_http

doc_dir = "data/carnegie"

fetch_archive_from_http(
    url="https://github.com/kynthesis/HaystackResearch/raw/main/how-to-win-friends-and-influence-people.zip",
    output_dir=doc_dir,
)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).
INFO:haystack.utils.import_utils:Fetching from https://github.com/kynthesis/HaystackResearch/raw/main/how-to-win-friends-and-influence-people.zip to 'data/carnegie'


True

# 5. Chuyển đổi các file sang định dạng Document

In [5]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor


converter = TextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_txt = converter.convert(file_path="data/carnegie/how-to-win-friends-and-influence-people.txt", meta=None)[0]

converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_pdf = converter.convert(file_path="data/carnegie/how-to-win-friends-and-influence-people.pdf", meta=None)[0]

converter = DocxToTextConverter(remove_numeric_tables=False, valid_languages=["en"])
doc_docx = converter.convert(file_path="data/carnegie/how-to-win-friends-and-influence-people.docx", meta=None)[0]

# 6. Tiền xử lý các Document

In [6]:
from haystack.nodes import PreProcessor


preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)
docs_default = preprocessor.process([doc_txt])
print(f"n_docs_input: 1\nn_docs_output: {len(docs_default)}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

n_docs_input: 1
n_docs_output: 1021
